In [28]:
from instagrapi import Client
import numpy as np
import pandas as pd
from collections import Counter
import itertools
import csv

In [2]:
# Логинимся в инстаграм через api
cl = Client()
cl.login("nast_ya5348", "12345432112344321")

True

In [4]:
user_id = cl.user_id_from_username("helpbaikal") # Достаем id Фонда

In [ ]:
# Обработка ДатаФрейма
df = pd.DataFrame(cl.user_followers(user_id))
data = df.T
data = data.rename(columns={0: 'id', 1: 'username', 2: 'name', 3: 'profile_pic_url', 5: 'is_private'})
data = data.drop(columns=[4, 6], axis=1).reset_index()
data = data.drop(columns=['id'], axis=1)
data['is_private'] = data['is_private'].apply(lambda x: str(x)[15:-1] == 'True')
data['username'] = data['username'].apply(lambda x: str(x)[14:-2])
data['name'] = data['name'].apply(lambda x: str(x)[15:-2])
data = data.loc[data['is_private'] == False]
data = data.reset_index(drop=True)

In [ ]:
# Создаем необходимы нам столбцы
data['user_following'] = np.nan
data['biography'] = np.nan
data['n_followers'] = np.nan
data['n_following'] = np.nan
data['n_media'] = np.nan
data['business_category'] = np.nan
data['category'] = np.nan

In [ ]:
# Здесь мы достаем все достпуные "username" подписок наших подписчиков
def ufget(id):
    try:
        user_fol = cl.user_following(id)
        user_fol = list(user_fol.values())
        user_fol = [i.username for i in user_fol]
        return user_fol
    except:
        return np.nan

In [ ]:
for i in range(0, 1551):
    uf = ufget(data['index'][i])
    data['user_following'][i] = uf
        
data.to_csv("output1.csv")

In [ ]:
# Здесь мы достаем всю доступную основную информаию о наших подписчиках
# Био, кол-во подписок, кол-во подписчиков, кол-во постов, название бизнес аккаунта (если есть), категория аккаунта

def dataget(username):
    try:
        return cl.user_info_by_username(username).dict()
    except:
        return np.nan

for i in range(0, 1551):
    personal_data = dataget(data['username'][i])
    if personal_data is not np.nan:
        data['biography'][i] = personal_data['biography']
        data['n_followers'][i] = personal_data['follower_count']
        data['n_following'][i] = personal_data['following_count']
        data['n_media'][i] = personal_data['media_count']
        data['business_category'][i] = personal_data['business_category_name']
        data['category'][i] = personal_data['category_name']
            
data.to_csv("output.csv")

In [ ]:
# Здесь мы достаем все хэштеги, которые используют в своих постах наши подписчики

def hashtagget(id):
    try:
        medias = cl.user_medias(id)
        medias = [medias[i].caption_text for i in range(len(medias))]
        medias = ' '.join(medias)
        line = word_tokenize(medias)
        hashs = [line[i+1] for i in range(len(line)) if line[i] =='#']
        return hashs
    except:
        return np.nan
    
for i in range(0, 1551):
    data['hashtags'][i] = hashtagget(data['index'][i])
    
data.to_csv("output.csv")

In [8]:
data

,index,username,name,profile_pic_url,is_private,hashtags,biography,n_followers,n_following,n_media,business_category,category,user_following
0,3048848177,tati_gemmakorea,GEMMA KOREA 🤍 ДЖЕММА ИРКУТСК,"('profile_pic_url', HttpUrl('https://scontent-...",False,"['бизнес', 'работаонлайн', 'джеммаиркутск', 'д...",@tati__khalmatova \nТвой надёжный бизнес-партн...,732.0,866.0,33.0,Creators & Celebrities,Personal Blog,"['dianada1592', 'im.sonya__', 'anna_zurbanova_..."
1,1568442186,natalyagankina,Наталья Ганькина,"('profile_pic_url', HttpUrl('https://scontent-...",False,"['уполномоченныйпоправамребенка', 'бурятия', '...",NaN,NaN,NaN,NaN,NaN,NaN,"['bair_buyantuev88', 'dobrodetel_064_priut', '..."
2,295823298,ilia_ogan,Ilia Oganisian,"('profile_pic_url', HttpUrl('https://scontent-...",False,"['АлександрКравченко', 'ПравительствоКалинингр...",HSE student 🎓 (Высшая Школа Экономики)\nDeterm...,1402.0,903.0,329.0,NaN,NaN,"['dariaryabchina', 'ninalebedeva', 'skhabibull..."
3,1392165160,sunnyolya,Olga Shakirova,"('profile_pic_url', HttpUrl('https://scontent-...",False,"['каре', 'оляsmmщик', 'пп', 'ЛАЙКчат', 'акселе...",A simple seeker of happiness,1125.0,557.0,319.0,NaN,Blogger,"['bina_dudkina', 'gennadycharaya', 'alexandrin..."
4,44767004511,viacheslavborodichh,Вячеслав Бородич,"('profile_pic_url', HttpUrl('https://instagram...",False,[],NaN,0.0,1.0,0.0,NaN,NaN,['helpbaikal']
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1546,1044541203,ahkulikova,ЛАЙФСТАЙЛ 🏆 БИЗНЕС,"('profile_pic_url', HttpUrl('https://scontent-...",False,"['статус', 'образование', 'семья', 'возраст', ...",💎 Из офисного планктона во фриланс\n💠10 лет в ...,962.0,425.0,331.0,Creators & Celebrities,Personal Blog,NaN
1547,222099204,nigoritsa.kate,Ekaterina Nigoritsa,"('profile_pic_url', HttpUrl('https://scontent-...",False,"['holapharmacie', 'гренаш', 'april', 'париж', ...",• Travel & Leisure 🌍\n• о еде и вине в stories,725.0,517.0,345.0,NaN,NaN,NaN
1548,4011615300,alisher_abdullaev_sh,Алишер Абдуллаев,"('profile_pic_url', HttpUrl('https://scontent-...",False,"['SWmoney', 'moscowbf2018', 'siberianhealth', ...",Выбираю партнёров по бизнесу. Дорого!#партнеры...,918.0,3126.0,57.0,Creators & Celebrities,Personal Blog,NaN
1549,3424568881,nastya_rogova.sw,Анастасия Рогова,"('profile_pic_url', HttpUrl('https://scontent-...",False,"['отдыхссемьей', 'experaltaplatinum', 'полюбис...",🤝Уважаю ЗОЖ\n🌺Знаю секрет вечной молодости😁\n...,233.0,198.0,67.0,Personal Goods & General Merchandise Stores,NaN,NaN


In [7]:
# Считываем наш готовый датасет

data = pd.read_csv('output.csv', index_col=[0])

Создадим словарь с самыми популярными подписками у наших подписчиков и отсортируем в убывающем порядке.

In [24]:
subdata_f = data[data['user_following'].apply(lambda x: str(x)) != 'nan']
subdata_f = subdata_f.reset_index(drop=True)

dict0 = list(set(subdata_f['user_following'][0][1:-1].split(',')))

for i in range(1, 872):
    dict0 += list(set(subdata_f['user_following'][i][1:-1].split(',')))
    
count0 = Counter(dict0)
count0 = dict(sorted(count0.items(), key=lambda item: -item[1])) 
dict(itertools.islice(count0.items(), 100)) # Выведем первые сто самых популярных результатов
# Полный список можно посмотреть в файле Following.csv

{" 'helpbaikal'": 859,
 " 'yurydud'": 192,
 " 'natgeo'": 177,
 " 'greenpeaceru'": 157,
 " 'navalny'": 122,
 " 'natgeotravel'": 122,
 " 'irina_hakamada'": 118,
 " 'siberianwellness'": 115,
 " 'ptencoff'": 113,
 " 'varlamov'": 112,
 " 'anton_lapenko'": 112,
 " 'labkovskiyofficial'": 110,
 " 'urgantcom'": 110,
 " 'pro.finansy'": 107,
 " 'doctor_komarovskiy'": 105,
 " 'zapovednoe_podlemorye'": 105,
 " 'xenia_sobchak'": 104,
 " 'wwfrussia'": 102,
 " 'natgeomagazineru'": 101,
 " 'maxgalkinru'": 100,
 " 'national_park_baikal'": 98,
 " 'baikal_save'": 95,
 " 'forbes.russia'": 94,
 " 'alexeymolchanov'": 94,
 " 'ikea_rus'": 93,
 " 'meduzapro'": 93,
 " 'siberianwellnessnews'": 90,
 " 'eco.irk'": 89,
 " 'pivo_varov'": 85,
 " 'garikkharlamov'": 82,
 " 'elena_blinovskaya'": 79,
 " 'irkutsk_news'": 79,
 " 'vecherniy_urgant'": 79,
 " 'tv_chernigovskaya'": 78,
 " 'nagiev.universal'": 78,
 " 'nasa'": 78,
 " 'gagara1987'": 78,
 " 'tretyakov_gallery'": 78,
 " 'pavelvolyaofficial'": 77,
 " 'dcim.ru'": 77,


In [30]:
with open('Following.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in count0.items():
       writer.writerow([key, value])

Создадим словарь с частотой упоминания хэштегов, повторения не учитываются, то есть если пользователь использовал хэштег более одного раза мы посчитаем это за одно использование. Цель в том, чтобы понять сколько человек хотя бы раз использовало тот или иной хэштег, так мы сможем понять основной круг интересов нашей аудитории и сделать некоторое иные выводы.

In [25]:
subdata = data[data['hashtags'].apply(lambda x: str(x)) != 'nan']
subdata = subdata.reset_index(drop=True)

dict1 = list(set(subdata['hashtags'][0][1:-1].split(',')))

for i in range(1, 1526):
    dict1 += list(set(subdata['hashtags'][i][1:-1].split(',')))

count1 = Counter(dict1)
count1 = dict(sorted(count1.items(), key=lambda item: -item[1]))
dict(itertools.islice(count1.items(), 100)) # Выведем первые сто самых популярных результатов
# Полный список можно посмотреть в файле Hashtags.csv

{'': 408,
 " 'лето'": 286,
 " 'красота'": 283,
 " '#'": 280,
 " 'природа'": 280,
 " 'весна'": 259,
 " 'байкал'": 255,
 " 'семья'": 241,
 " 'осень'": 237,
 " 'москва'": 236,
 " 'здоровье'": 223,
 " 'новыйгод'": 223,
 " 'сибирскоездоровье'": 217,
 " 'отдых'": 215,
 " 'зима'": 215,
 " 'siberianwellness'": 208,
 " 'любовь'": 207,
 " 'спорт'": 195,
 " 'путешествие'": 192,
 " 'дети'": 190,
 " 'россия'": 190,
 " 'сибирь'": 189,
 " 'счастье'": 188,
 " 'зож'": 187,
 " 'горы'": 184,
 " 'море'": 182,
 " 'Repost'": 182,
 " 'путешествия'": 180,
 " 'nature'": 177,
 " 'иркутск'": 176,
 " 'экология'": 172,
 " '8марта'": 168,
 " 'деньрождения'": 168,
 " 'праздник'": 166,
 " 'siberianhealth'": 165,
 " 'baikal'": 161,
 " 'цветы'": 160,
 " 'лес'": 160,
 " 'russia'": 157,
 " 'солнце'": 156,
 " 'подарки'": 153,
 " 'закат'": 152,
 " 'travel'": 151,
 " 'love'": 150,
 " 'бизнес'": 147,
 " 'Байкал'": 147,
 " 'снег'": 143,
 " 'друзья'": 142,
 " 'summer'": 138,
 " 'beauty'": 135,
 " 'жизнь'": 135,
 " 'деньпобеды'

In [29]:
with open('Hashtags.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in count1.items():
       writer.writerow([key, value])